
#Importing libraries and working on data


In [33]:
!pip install vecstack

In [34]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from collections import Counter #for Smote,

import warnings
warnings.filterwarnings("ignore")

In [35]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [51]:
trainfile = r'/gdrive/MyDrive/ML/Assignment3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/MyDrive/ML/Assignment3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head())

(65000, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16               2

In [52]:
print(test_data.shape)
print(test_data.head())

(173836, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                13                22                4                4   
1                 4                 5                8               14   
2                 3                 3               11               18   
3                 5                 9               14               22   
4                12                21                4                5   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0                4                4                3                3   
1                8               14                7               12   
2               11               18               10               16   
3               15               22               13               20   
4                4                5                4                4   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0                3               

In [53]:
train_cat_cols = train_data.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)
test_cat_cols = test_data.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(test_cat_cols.shape)
print(test_cat_cols)

(0,)
Index([], dtype='object')
(1,)
Index(['GeographicField64'], dtype='object')


In [54]:
test_data=test_data.drop(columns=["GeographicField64"])

In [55]:
train_data.isnull().sum()


CoverageField11A        0
CoverageField11B        0
CoverageField1A         0
CoverageField1B         0
CoverageField2A         0
                       ..
GeographicField64_CA    0
GeographicField64_IL    0
GeographicField64_NJ    0
GeographicField64_TX    0
QuoteConversion_Flag    0
Length: 596, dtype: int64

In [56]:
test_data.isnull().sum()

CoverageField11A        0
CoverageField11B        0
CoverageField1A         0
CoverageField1B         0
CoverageField2A         0
                       ..
GeographicField63_Y     0
GeographicField64_CA    0
GeographicField64_IL    0
GeographicField64_NJ    0
GeographicField64_TX    0
Length: 595, dtype: int64

In [57]:

#Set dependent and independent variables
#Select dependent variables Column
X_train = train_data.iloc[:, :-1]
X_test = test_data.iloc[:, :]


#Select just Target Column
y_train = train_data.iloc[:, -1]


print(X_train.shape)
print(X_test.shape)

print(y_train.shape)


(65000, 595)
(173836, 595)
(65000,)


In [58]:
print(train_data.head())

   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16               22  ...       

# DT Classifier

In [59]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)

In [60]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_DT=pd.DataFrame(clf.predict(X_test),columns=["Prediction"] )

pred_DT.head()

,Prediction
0,0
1,1
2,0
3,0
4,1


In [61]:
#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_DT_Probability =pd.DataFrame(clf.predict_proba(X_test))

pred_DT_Probability.head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [63]:
#Write into a file with actual prediction and corresponding probability
res = pd.concat([pred_DT,pred_DT],axis=1)
#.to_csv("/gdrive/MyDrive/ML/DT3_results.csv", index = None)
res.head()

,Prediction,Prediction
0,0,0
1,1,1
2,0,0
3,0,0
4,1,1


# RF Classifier

In [64]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)

In [65]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_RF=pd.DataFrame(rfc.predict(X_test),columns=["Prediction"] )

pred_RF.head()

,Prediction
0,0
1,0
2,0
3,0
4,0


In [66]:

pred_RF.head()
#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_RF_Probability =pd.DataFrame(rfc.predict_proba(X_test))

pred_RF_Probability.head()

,0,1
0,0.93,0.07
1,0.87,0.13
2,0.93,0.07
3,0.96,0.04
4,0.73,0.27


In [67]:

pred_RF_Probability.head()
#Write into a file with actual prediction and corresponding probability
res=pd.concat([pred_RF,pred_RF_Probability],axis=1)
#.to_csv("/gdrive/MyDrive/Colab Notebooks/ML/RF3_results.csv", index = None)

# res=pd.read_csv('/gdrive/MyDrive/Colab Notebooks/ML/RF3_results.csv')
res.head()


,Prediction,0,1
0,0,0.93,0.07
1,0,0.87,0.13
2,0,0.93,0.07
3,0,0.96,0.04
4,0,0.73,0.27


# GradientBoosting Classifier

In [68]:
#Gradient Boosting ============================================================
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)

In [69]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_abc=pd.DataFrame(abc.predict(X_test),columns=["Prediction"] )

#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_abc_Probability =pd.DataFrame(abc.predict_proba(X_test))

#Write into a file with actual prediction and corresponding probability
#pd.concat([pred_abc,pred_abc_Probability],axis=1).to_csv("/gdrive/MyDrive/Colab Notebooks/ML/GB3_results.csv", index = None)

res=pd.concat([pred_abc,pred_abc_Probability],axis=1)
#.to_csv("/gdrive/MyDrive/Colab Notebooks/ML/GB3_results.csv", index = None)

# res=pd.read_csv('/gdrive/MyDrive/Colab Notebooks/ML/GB3_results.csv')
res.head()


,Prediction,0,1
0,0,0.988995,0.011005
1,0,0.938972,0.061028
2,0,0.940773,0.059227
3,0,0.961426,0.038574
4,0,0.817220,0.182780


# KNN Classifier

In [73]:
#Construct K-nearest Neighbour classifier
# Default mode
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predict=knn.predict(X_test)

In [74]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_knn=pd.DataFrame(knn.predict(X_test),columns=["Prediction"] )


#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_knn_Probability =pd.DataFrame(knn.predict_proba(X_test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_knn,pred_knn_Probability],axis=1).to_csv("/gdrive/MyDrive/ML/Assignment3/KNN_Results.csv", index = None)

res=pd.read_csv('/gdrive/MyDrive/ML/Assignment3/KNN_Results.csv')
res.head()

,Prediction,0,1
0,0,0.8,0.2
1,0,0.8,0.2
2,0,0.6,0.4
3,0,1.0,0.0
4,0,0.6,0.4


# Logistic Classifier

In [ ]:
#Construct Logistic regression classifier
lgrg = LogisticRegression()
lgrg.fit(X_train, y_train)
lgrg_predict=lgrg.predict(X_test)

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_lgrg=pd.DataFrame(lgrg.predict(X_test),columns=["Prediction"] )


#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_lgrg_Probability =pd.DataFrame(lgrg.predict_proba(X_test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_lgrg,pred_lgrg_Probability],axis=1).to_csv("gdrive/MyDrive/ML/Assignment3/LogReg3_results.csv", index = None)

res=pd.read_csv('/gdrive/MyDrive/ML/Assignment3/LogReg3_results.csv')
res.head()

# MLP Classifier

In [81]:

#Construct Multilayer Perceptron using the best parameters
#Default mode
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
mlp_predict=mlp.predict(X_test)

In [82]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_mlp=pd.DataFrame(mlp.predict(X_test),columns=["Prediction"] )

pred_mlp.head()

#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_mlp_Probability =pd.DataFrame(mlp.predict_proba(X_test))

pred_mlp_Probability.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([pred_mlp,pred_mlp_Probability],axis=1).to_csv("gdrive/MyDrive/ML/Assignment3/MLP3_results.csv", index = None)

res=pd.read_csv('gdrive/MyDrive/ML/Assignment3/MLP3_results.csv')
res.head()

OSError: ignored

In [ ]:
# #Construct Support Vector Machines using the best parameters
# #Default mode
# svc = svm.SVC()
# svc.fit(X_train, y_train)
# svc_predict=svc.predict(X_test)


In [ ]:
# model_svc = LinearSVC()
# m_svm = CalibratedClassifierCV(model_svc)
# m_svm.fit(X_train, y_train)
# m_svm_predict=m_svm.predict(X_test)

In [ ]:
# #Testset prediction======================================================================
# #Get Class Prediction as a data frame with header as Prediction
# pred_svc=pd.DataFrame(m_svm.predict(X_test),columns=["Prediction"] )


# #Get Class Prediction probabilities as a data frame
# #Get Prediction Probability for the predicted class as a dataframe
# pred_svc_Probability =pd.DataFrame(m_svm.predict_proba(X_test))


# #Write into a file with actual prediction and corresponding probability
# pd.concat([pred_svc,pred_svc_Probability],axis=1).to_csv("/gdrive/MyDrive/Colab Notebooks/ML/SVM3_results.csv", index = None)

# res=pd.read_csv('/gdrive/MyDrive/Colab Notebooks/ML/SVM3_results.csv')
# res.head()

# SMOTE and Stacking

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
# print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, KNeighborsClassifier, LogisticRegression RandomForest and Decision Tree Classifier\n")

models = [ RandomForestClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier(), KNeighborsClassifier(), LogisticRegression()]

S_Train, S_Test = stacking(models,
                           X_res, y_res, X_test,
                           regression=False,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           metric=accuracy_score,

                           n_folds=4,

                           stratified=True,

                           shuffle=True,

                           random_state=0,

                           verbose=2)

# Best Model Hypertuning and Results

In [ ]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()

model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_model=pd.DataFrame(model.predict(S_Test),columns=["Prediction"] )

#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_model_Probability =pd.DataFrame(model.predict_proba(S_Test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_model,pred_model_Probability],axis=1).to_csv("/gdrive/MyDrive/ML/Assignment3/Final_Model2_results.csv", index = None)

res=pd.read_csv('/gdrive/MyDrive/ML/Assignment3/Final_Model2_results.csv')
res.head()

In [ ]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
model =GradientBoostingClassifier()

model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)


#Randomized Search for hyperparameter tuning
model_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
model_random.fit(X_train, y_train)
grid_parm_model=model_random.best_params_
print(grid_parm_model)

In [ ]:
#STACKING - CONSTRUCT A GRADIENT BOOSTING MODEL using best parameter=============================
model_hpt =GradientBoostingClassifier(n_estimators= 50, learning_rate=0.1,)

model_hpt = model_hpt.fit(S_Train, y_res)
y_pred = model_hpt.predict(S_Test)

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_model_hpt=pd.DataFrame(model_hpt.predict(S_Test),columns=["Prediction"] )

#Get Class Prediction probabilities as a data frame
#Get Prediction Probability for the predicted class as a dataframe
pred_model_hpt_Probability =pd.DataFrame(model_hpt.predict_proba(S_Test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_model_hpt,pred_model_hpt_Probability],axis=1).to_csv("/gdrive/MyDrive/ML/Assignment3/Final_Model_HPT2_results.csv", index = None)

res=pd.read_csv('/gdrive/MyDrive/ML/Assignment3/Final_Model_HPT2_results.csv')
res.head()

In [ ]:
id = test_data['QuoteNumber']
print(id)

In [ ]:
model_op = model_hpt.predict_proba(S_Test)
final_df = pd.DataFrame(model_op[:,1],columns=["QuoteConversion_Flag"])



In [ ]:
result = pd.concat([id,final_df],axis=1)
result

In [ ]:
result.to_csv("/gdrive/MyDrive/ML/Assignment3/Final_Model_HPT2_results.csv", index = None)